# 1. IMports

In [1]:
from gensim.models import LdaModel
from gensim.corpora import Dictionary
import pickle
import pandas as pd

from nltk.tokenize import word_tokenize

# from custom script in this dir
from prepro_script import text_preprocessing_id

In [2]:
df = pd.read_csv('labeled_documents.csv')
df

,rating,reviews,company,province,parsed_date,processed_reviews,tokenized_reviews,topic
0,1,"Bad service given to customers, my own experie...",JNE,Aceh,2021-01-21,layan buruk langgan alam ambil jasa kirim kelu...,"['layan', 'buruk', 'langgan', 'alam', 'ambil',...",Delay/ Lambat Pengiriman
1,1,Too slow,JNE,Aceh,2016-01-23,lambat,['lambat'],Delay/ Lambat Pengiriman
2,2,Slow service,JNE,Aceh,2022-01-21,lambat,['lambat'],Delay/ Lambat Pengiriman
3,1,Bad service,JNE,Aceh,2019-01-22,buruk,['buruk'],Delay/ Lambat Pengiriman
4,1,The courier was too slow for many reasons even...,JNE,Aceh,2025-01-06,kurir lambat kira pakai sayang m,"['kurir', 'lambat', 'kira', 'pakai', 'sayang',...",Delay/ Lambat Pengiriman
...,...,...,...,...,...,...,...,...
13922,1,Lambat,Sicepat,Sumatera Utara,2021-01-21,lambat,['lambat'],Delay/ Lambat Pengiriman
13923,1,Kargo Silambat,Sicepat,Sumatera Utara,2021-01-21,kargo lambat,"['kargo', 'lambat']",Delay/ Lambat Pengiriman
13924,1,SILELET CEMBRETTTTTT,Sicepat,Sumatera Utara,2021-01-21,lambat cembret,"['lambat', 'cembret']",Delay/ Lambat Pengiriman
13925,1,Lambat sekali....,Sicepat,Sumatera Utara,2021-01-21,lambat,['lambat'],Delay/ Lambat Pengiriman


# 2. Infer


## a. Jelek

In [2]:
# Load the saved LDA model
jelek_lda_model = LdaModel.load('jelek_lda_model.model')

# Load the dictionary
jelek_dictionary = Dictionary.load('jelek_lda_dictionary.dict')

# Load the corpus (optional, for validation purposes)
with open('jelek_lda_corpus.pkl', 'rb') as f_jelek:
    jelek_corpus = pickle.load(f_jelek)

print("Model, dictionary, and corpus loaded successfully.")

Model, dictionary, and corpus loaded successfully.


In [3]:
jelek_topic_labels = {
    0: "Pelayan Buruk",
    1: "Delay / Lambat",
    2: "Miskomunikasi Kurir"
}

### - Single infer

In [4]:
# Single review
jelek_new_review = "Paket saya terlambat dan layanan buruk sekali."

In [5]:
# Preprocess the review
jelek_processed_review = await text_preprocessing_id(jelek_new_review)

In [6]:

# Convert to Bag of Words
bow_vector = jelek_dictionary.doc2bow(jelek_processed_review.split())

# Get topic distribution with labels
topics = jelek_lda_model.get_document_topics(bow_vector, minimum_probability=0.0)

# Display topics with labels
print(f"Review : {jelek_new_review}")
print(f"Processed Review: {jelek_processed_review}")
print()
print("Inferred Topics with Labels:")
for topic_id, prob in topics:
    print(f"  {jelek_topic_labels[topic_id]}: {prob:.2%}")

Review : Paket saya terlambat dan layanan buruk sekali.
Processed Review: lambat buruk

Inferred Topics with Labels:
  Pelayan Buruk: 35.21%
  Delay / Lambat: 47.88%
  Miskomunikasi Kurir: 16.91%


### - Batch Infer

In [29]:
# batch reviews
jelek_new_reviews = [
    "Paket saya terlambat dan layanan buruk sekali.",
    "Opo toh iki cok. Bangsat. Barangku katut ndi cok. Kntl. Bubar ae. Gak guna!",
    "Kurir sangat tidak sopan dang ngawur. Masak iya kirim tv di taruh di luar pagar rumah. Mengecewakan!",
    "JNE pungo, hana utak, hanjeut meupike gob rugo, lage bui…😂😂😂",
    """Dima kendala sanak.....lamo bana paket ko..
    Jauh bana simpang haru ko....dari pagi tadi lai....baru jam 12 ba anta paket....sampai kini nda juo tibo2 ala jam 2 lewat lo
    Deh mande mak oi.......""",
    """Ndk sasuai jo namo, paket berhari2 di padang ndk dikirim ka ulak karang, sicepat a namoe tu
    Ancak tutuik se lai""",
    "Iko cocok namonyo diganti jadi SI PANTEK, alamat alah jaleh bakecekan bad adress, sudah tu barang di return baliak, eee kapunduang"
]

In [30]:
for review in jelek_new_reviews:
    processed_review = await text_preprocessing_id(review)
    bow_vector = jelek_dictionary.doc2bow(processed_review.split())
    topics = jelek_lda_model.get_document_topics(bow_vector, minimum_probability=0.0)
    print(f"Review: {review}")
    print(f"Processed Review : {processed_review}")
    print("Inferred Topics with Labels:")
    for topic_id, prob in topics:
        print(f"  {jelek_topic_labels[topic_id]}: {prob:.2%}")
    print()

Review: Paket saya terlambat dan layanan buruk sekali.
Processed Review : lambat buruk
Inferred Topics with Labels:
  Pelayan Buruk: 35.21%
  Delay / Lambat: 47.88%
  Miskomunikasi Kurir: 16.91%

Review: Opo toh iki cok. Bangsat. Barangku katut ndi cok. Kntl. Bubar ae. Gak guna!
Processed Review : barang bagus bubar guna
Inferred Topics with Labels:
  Pelayan Buruk: 19.24%
  Delay / Lambat: 57.63%
  Miskomunikasi Kurir: 23.13%

Review: Kurir sangat tidak sopan dang ngawur. Masak iya kirim tv di taruh di luar pagar rumah. Mengecewakan!
Processed Review : sopan dang sembarang masak tv taruh pagar rumah kecewa
Inferred Topics with Labels:
  Pelayan Buruk: 20.61%
  Delay / Lambat: 35.18%
  Miskomunikasi Kurir: 44.20%

Review: JNE pungo, hana utak, hanjeut meupike gob rugo, lage bui…😂😂😂
Processed Review : otak mikir orang kalah kaya narapidana
Inferred Topics with Labels:
  Pelayan Buruk: 26.10%
  Delay / Lambat: 44.19%
  Miskomunikasi Kurir: 29.71%

Review: Dima kendala sanak.....lamo bana

## b. Bagus